# Carga de pesos preentrenados desde OpenAI

En secciones anteriores, se entrenó un pequeño modelo GPT-2   con  un  conjunto  de  datos  limitado  que  incluía  un  libro  de  cuentos.  
Afortunadamente,  OpenAI  compartió  abiertamente  los  pesos  de  sus  modelos  GPT2,  eliminando  así  la  necesidad  de  invertir  decenas  a  cientos  de  miles  de  dólares  en  volver  a  entrenar  el  modelo  en  un  corpus  grande  nosotros  mismos.

Durante esta sección se cargarán los pesos en la clase GPTModel.  Aquí,  los  pesos  se  refieren  a  los  parámetros  de  peso  almacenados  en  los  atributos .weight  de  las  capas  lineales  y  de  incrustación  de  PyTorch ,  por  ejemplo: Se accedio  a  ellos  anteriormente  mediante  model.parameters()  durante  el  entrenamiento  del  modelo.

OpenAI  guardó  originalmente  los  pesos  GPT2  mediante  TensorFlow.  Además,  el  siguiente  código  usará  una  herramienta  de  barra  de  progreso  llamada  tqdm  para  rastrear  el  proceso  de  descarga.

**pip install tensorflow>=2.15.0  tqdm>=4.66**

In [1]:
import urllib.request
url = (
    "https://raw.githubusercontent.com/rasbt/"
    "LLMs-from-scratch/main/ch05/"
    "01_main-chapter-code/gpt_download.py"
)
filename = url.split('/')[-1]
urllib.request.urlretrieve(url, filename)

('gpt_download.py', <http.client.HTTPMessage at 0x26e9d2755d0>)

Dado que el código de descarga es demasiado largo, se ha optado por descargar el módulo python que e encargará de todas esta tareas.
Ahora  se puede importar  la  función  download_and_load_gpt2  del  archivo  gpt_download.py  de  la  siguiente  manera,  que  cargará  las  configuraciones  de  la  arquitectura  GPT2  (settings)  y  los  parámetros  de  peso  (params)  en la sesión  de  Python:

In [2]:
from gpt_download import download_and_load_gpt2
settings, params = download_and_load_gpt2(model_size="124M", models_dir="gpt2")

checkpoint: 100%|██████████| 77.0/77.0 [00:00<00:00, 77.0kiB/s]
encoder.json: 100%|██████████| 1.04M/1.04M [00:01<00:00, 899kiB/s]
hparams.json: 100%|██████████| 90.0/90.0 [00:00<?, ?iB/s]
model.ckpt.data-00000-of-00001: 100%|██████████| 498M/498M [06:50<00:00, 1.21MiB/s]   
model.ckpt.index: 100%|██████████| 5.21k/5.21k [00:00<00:00, 5.21MiB/s]
model.ckpt.meta: 100%|██████████| 471k/471k [00:00<00:00, 574kiB/s] 
vocab.bpe: 100%|██████████| 456k/456k [00:00<00:00, 646kiB/s] 


Una vez completada la ejecución del código anterior, sería necesario inspenccionar el contenido de las configuraciones y parámetros.

In [3]:
print("Settings:", settings)
print("Parameter dictionary keys:", params.keys())

Settings: {'n_vocab': 50257, 'n_ctx': 1024, 'n_embd': 768, 'n_head': 12, 'n_layer': 12}
Parameter dictionary keys: dict_keys(['blocks', 'b', 'g', 'wpe', 'wte'])


Tanto  los  parámetros  como  las  configuraciones  son  diccionarios  de  Python.  El  diccionario  de  configuraciones  almacena  la  configuración  de  la  arquitectura  LLM  de  forma  similar  a  la  configuración  GPT_CONFIG_124M  definida  manualmente .  El  diccionario  de  parámetros  contiene  los  tensores  de  peso. Imprimir  el  contenido  de  los  pesos  ocuparía  demasiado  espacio  en  la  pantalla.  Sin  embargo,  se puede  inspeccionar  estos  tensores  imprimiendo  el  diccionario  completo  mediante  print(params)  o  seleccionando  tensores  individuales  mediante  las  claves  correspondientes,  por  ejemplo,  los  pesos  de  la  capa  de  incrustación.

In [4]:
print(params["wte"])
print("Token embedding weight tensor dimensions:", params["wte"].shape)

[[-0.11010301 -0.03926672  0.03310751 ... -0.1363697   0.01506208
   0.04531523]
 [ 0.04034033 -0.04861503  0.04624869 ...  0.08605453  0.00253983
   0.04318958]
 [-0.12746179  0.04793796  0.18410145 ...  0.08991534 -0.12972379
  -0.08785918]
 ...
 [-0.04453601 -0.05483596  0.01225674 ...  0.10435229  0.09783269
  -0.06952604]
 [ 0.1860082   0.01665728  0.04611587 ... -0.09625227  0.07847701
  -0.02245961]
 [ 0.05135201 -0.02768905  0.0499369  ...  0.00704835  0.15519823
   0.12067825]]
Token embedding weight tensor dimensions: (50257, 768)


Se descargan  y  cargan  los  pesos  del  modelo  GPT2  más  pequeño  mediante  la  configuración  `  download_and_load_gpt2(model_size="124M", ...)  `.  Sin  embargo,  tenga  en  cuenta  que  OpenAI  también  comparte  los  pesos  de  los  modelos  más  grandes:  "355M",  "774M"  y  "1558M".  La  arquitectura  general  de  estos  modelos  GPT  de  diferentes  tamaños  es  la  misma.

![Texto alternativo](./imgs/5.15.png)

Como  se  ilustra  en  la  Figura, la  arquitectura  general  de  los  modelos  GPT2  de  diferentes  tamaños  se  mantiene  igual,  salvo  que  los  distintos  elementos arquitectónicos  se  repiten  con  diferente  frecuencia  y  el  tamaño  de  la  incrustación  varía. 
Después  de  cargar  los  pesos  del  modelo  GPT2  en  Python,  todavía  es necesario  transferirlos  desde los  diccionarios  de  configuraciones  y  parámetros  en  la instancia  GPTModel.

In [7]:
#Primero se crea un diccionario que lista las diferencias entre los diferentes modelos GPT.

model_configs = {
    "gpt2-small (124M)": {"emb_dim": 768, "n_layers": 12, "n_heads": 12},
    "gpt2-medium (355M)": {"emb_dim": 1024, "n_layers": 24, "n_heads": 16},
    "gpt2-large (774M)": {"emb_dim": 1280, "n_layers": 36, "n_heads": 20},
    "gpt2-xl (1558M)": {"emb_dim": 1600, "n_layers": 48, "n_heads": 25},
}

Suponer  que   interesa  cargar  el  modelo  más  pequeño,  "gpt2small  (124M)".  Se puede  usar  configuraciones  correspondientes  de  la  tabla  model_configs  capaces  de  actualizar  la  longitud  completa GPT_CONFIG_124M.

In [5]:
import sys
import os

# Obtiene la ruta de la carpeta principal del proyecto (subiendo un nivel desde seccion05)
ruta_proyecto_principal = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Añade esta ruta a la lista de lugares donde Python busca módulos
if ruta_proyecto_principal not in sys.path:
    sys.path.append(ruta_proyecto_principal)

In [8]:
import torch
import torch.nn as nn
import tiktoken
from seccion04_ImplementacionGPTGeneracionTexto.gptModel import GPTModel
from seccion04_ImplementacionGPTGeneracionTexto.gptConfig124M import GPT_CONFIG_124M
from seccion02_TrabajarDatosTexto.dataloader_v1 import create_dataloader_v1
from trainModelSimple import train_model_simple

model_name = "gpt2-small (124M)"
NEW_CONFIG = GPT_CONFIG_124M.copy()
NEW_CONFIG.update(model_configs[model_name])


OpenAI  utilizó  vectores  de  sesgo  en  las  capas  lineales  del  módulo  de  atención  multicabezal  para  implementar  los  cálculos  de  las  matrices  de  consulta,  clave  y  valor.  Los  vectores  de  sesgo  ya  no  se  utilizan  comúnmente  en  los  LLM,  ya  que  no  mejoran  el  rendimiento  del  modelado  y,  por  lo  tanto,  son  innecesarios.  Sin  embargo,  dado  que  se está trabajando con  pesos  preentrenados,  se necesita  ajustar  la  configuración  para  garantizar  la  consistencia  y  habilitar  estos  vectores  de  sesgo:

In [9]:
NEW_CONFIG.update({"qkv_bias": True})
gpt = GPTModel(NEW_CONFIG)
gpt.eval()

GPTModel(
  (tok_emb): Embedding(50257, 768)
  (pos_emb): Embedding(1024, 768)
  (drop_emb): Dropout(p=0.1, inplace=False)
  (trf_blocks): Sequential(
    (0): TransformerBlock(
      (att): MultiHeadAttention(
        (W_query): Linear(in_features=768, out_features=768, bias=True)
        (W_key): Linear(in_features=768, out_features=768, bias=True)
        (W_value): Linear(in_features=768, out_features=768, bias=True)
        (out_proj): Linear(in_features=768, out_features=768, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (ff): FeedForward(
        (layers): Sequential(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU()
          (2): Linear(in_features=3072, out_features=768, bias=True)
        )
      )
      (norm1): LayerNorm()
      (norm2): LayerNorm()
      (drop_shortcut): Dropout(p=0.1, inplace=False)
    )
    (1): TransformerBlock(
      (att): MultiHeadAttention(
        (W_query): Linear(in_features=7

De  forma  predeterminada,  la  instancia  de  GPTModel  se  inicializa  con  pesos  aleatorios  para  el  preentrenamiento.  

El  último  paso  para  usar  los  pesos  del  modelo  de  OpenAI  es  sobrescribir  estos  pesos  aleatorios  con  los  pesos  cargados  en  el  diccionario  de  parámetros.

Para  esto,  primero  se define  una  pequeña  función  de  utilidad  de  asignación  que  verifica  si  dos  tensores  o  matrices  (izquierdo  y  derecho)  tienen  las  mismas  dimensiones  o  forma  y  devuelve  el  tensor  correcto  como  parámetros  entrenables  de  PyTorch: